In [ ]:
#@title Get Loaded into the Pipeline
try:
    from boxsdk import OAuth2, Client
except ImportError:
    %pip install boxsdk --quiet
    from boxsdk import OAuth2, Client
import os, time, shutil
from google.colab import drive
import pandas as pd
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
from IPython.display import clear_output, Audio, display
class BoxNavigator():
    def __init__(self) -> 'None':
        drive.mount('/content/drive/', force_remount=True)
        self.home_dir = '/content/drive/MyDrive/manacus_v1/'
        system_files_dir = os.path.join(self.home_dir, 'system files')
        if not os.path.exists(system_files_dir):
            self.setup_credentials()
        else:
            self.load_credentials_from_file()

        if not os.path.exists(self.home_dir):
            os.makedirs(self.home_dir)
            print(f"New folder made: {self.home_dir}.")

    def load_credentials_from_file(self) -> 'None':
        credentials_file_path = os.path.join(self.home_dir, 'system files', 'box_credentials.txt')
        with open(credentials_file_path, 'r') as file:
            lines = file.readlines()
            self.client_id = lines[0].strip()
            self.client_secret = lines[1].strip()
            self.access_token = lines[2].strip()

        self.auth = OAuth2(client_id=self.client_id, client_secret=self.client_secret, access_token=self.access_token)
        self.client = Client(self.auth)

    def setup_credentials(self) -> 'None':
        print("Login to https://tulane.app.box.com/developers/console and paste the following:")
        self.client_id = input("Enter your Box Client ID: ")
        self.client_secret = input("Enter your Box Client Secret: ")
        self.access_token = input("Enter your Box Developer Token: ")
        sys_files_path = os.path.join(self.home_dir, 'system files')
        os.makedirs(sys_files_path)
        os.mkdir(os.path.join(self.home_dir, 'Downloaded Videos'))
        credentials_file_path = os.path.join(sys_files_path, 'box_credentials.txt')
        with open(credentials_file_path, 'w') as file:
            file.write(f"{self.client_id}\n{self.client_secret}\n{self.access_token}")

        self.auth = OAuth2(client_id=self.client_id, client_secret=self.client_secret, access_token=self.access_token)
        self.client = Client(self.auth)

    def search(self, video_name: str):
      try:
        return next(iter(self.client.search().query(query=video_name, limit=1)))
      except:
        self.access_token = input("Enter your Box Developer Token: ")
        sys_files_path = os.path.join(self.home_dir, 'system files')
        credentials_file_path = os.path.join(sys_files_path, 'box_credentials.txt')
        with open(credentials_file_path, 'w') as file:
            file.write(f"{self.client_id}\n{self.client_secret}\n{self.access_token}")
        self.auth = OAuth2(client_id=self.client_id, client_secret=self.client_secret, access_token=self.access_token)
        self.client = Client(self.auth)
        return self.search(video_name)

    def download_vid(self, video_name: str) -> None:
        assert os.path.exists(os.path.join(self.home_dir, 'Downloaded Videos'))
        video_path = os.path.join(self.home_dir, 'Downloaded Videos', video_name)
        if os.path.exists(video_path):
            print(f"Video '{video_name}' already exists in Drive.")
        else:
            video_item_returned = self.search(video_name)
            if video_item_returned.name == video_name and video_item_returned.type == 'file' and video_name.lower().endswith("mp4"):
                print(f"Downloading video '{video_name}' to Drive: {video_path}...")
                with open(video_path, 'wb') as f:
                    self.client.file(video_item_returned.id).download_to(f)
                print(f"    Successfully saved video '{video_name}' from Box in Drive.")
            else:
                print(f"    Video '{video_name}' not found in Box or is not an mp4 file.")

    def extract_audio_clips(self, video_name):
        videos_folder_path = os.path.join(self.home_dir, 'Downloaded Videos')
        video_path = os.path.join(videos_folder_path, video_name)
        video_files = [f for f in [video_name] if os.path.isfile(video_path) and (f.lower().endswith(".mp4") or f.lower().endswith(".avi"))]

        for video_file in video_files:
            video_path = os.path.join(videos_folder_path, video_file)

            try:
                # Create directory for audio clips of this video
                video_name_without_ext = os.path.splitext(video_file)[0]
                audio_clips_folder_path = os.path.join(videos_folder_path, video_name_without_ext)

                if not os.path.exists(audio_clips_folder_path):
                    os.makedirs(audio_clips_folder_path)
                    video_clip = VideoFileClip(video_path)
                    audio_clip = video_clip.audio

                    audio_clip_duration = int(audio_clip.duration)  # Duration in seconds

                    i = 0
                    while i < audio_clip_duration:
                        start_time = i
                        end_time = i + 0.5
                        subclip = audio_clip.subclip(start_time, end_time)

                        # Extract audio subclip and save
                        clip_filename = f"{video_name_without_ext}_{int(i*2):04d}.wav"
                        clip_filepath = os.path.join(audio_clips_folder_path, clip_filename)

                        if not os.path.exists(clip_filepath):
                            subclip.write_audiofile(clip_filepath, codec='pcm_s16le')
                            print(f"Saved clip: {clip_filename}")

                        i += 0.5

                    video_clip.reader.close()
                    audio_clip.reader.close_proc()
            return audio_clips_folder_path

            except IOError as e:
                print(f"An error occurred for {video_file}: {e}")

    def labelclips(self, video_name):
        # Check if labels folders exist, if not, create them
        labels_dir = os.path.join(self.home_dir, 'labels')
        labels = ['s', 'g', 'c', 'c2', 'r', 'w', 'n', 'q']  # Define your labels here
        label_dirs = [os.path.join(labels_dir, label) for label in labels]
        for label_dir in label_dirs:
            if not os.path.exists(label_dir):
                os.makedirs(label_dir)

        # Get the directory that contains the audio clips
        audio_clips_dir = os.path.join(self.home_dir, 'Downloaded Videos', os.path.splitext(video_name)[0])
        if os.path.exists(audio_clips_dir):
            audio_clips = [clip for clip in os.listdir(audio_clips_dir) if clip.lower().endswith(".wav")]
            n_clips = len(audio_clips)
            for audio_clip in audio_clips:
                n_clips-=1
                clear_output()
                # Play audio clip
                clip_path = os.path.join(audio_clips_dir, audio_clip)
                print(f"Playing {audio_clip}")
                display(Audio(clip_path, autoplay=True))
                #time.sleep(1)

                # Ask for the user input for labeling
                label = ''
                print(f"{n_clips} clips remaining.")
                while label not in labels:
                    time.sleep(1)
                    label = input("Enter label (s, g, c, c2, r, w, n, q):\n").lower().strip()
                # Move the audio clip to the corresponding label directory
                destination_path = os.path.join(labels_dir, label, audio_clip)
                shutil.move(clip_path, destination_path)
                print(f"Moved {audio_clip} to {destination_path}")
        else:
            print(f"There are no audio clips extracted for the video {video_name}.")

nav = BoxNavigator()
print('Please upload a spreadsheet to My Drive/manacus3/spreadsheets/')
sheets_folder = '/content/drive/My Drive/manacus3/spreadsheets/'
os.makedirs(sheets_folder, exist_ok=True)
sheet_paths = os.listdir(sheets_folder)
while len(sheet_paths) == 0:
  drive.mount('/content/drive/', force_remount=True)
  sheet_paths = os.listdir(sheets_folder)
  clear_output()
  time.sleep(2)
  input('Have you uploaded a spreadsheet to My Drive/manacus3/spreadsheets/? (Y/N)')
spreadsheet = pd.read_excel(os.path.join(sheets_folder, sheet_paths[0]))
labeling_for = 'Calls'
spreadsheet['Calls'] = pd.to_numeric(spreadsheet['Calls'], errors='coerce')
spreadsheet = spreadsheet.dropna(subset=['Calls'])
spreadsheet = spreadsheet.sort_values(by=labeling_for, ascending=False).reset_index(drop=True)
print(f'Labeling for {labeling_for}')

SyntaxError: expected 'except' or 'finally' block (<ipython-input-1-426b748324b3>, line 121)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
from IPython.display import Audio, display

def string_to_dataframe(s):
    try:
        s = s.replace("'", '"')
        data = json.loads(s)
        df = pd.DataFrame(data)
        return df
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Load the CSV file
not_present = pd.read_csv('/content/log (59).csv')
i = 19
video = not_present['predictions'].iloc[i]

# Convert the 'predictions' string to a DataFrame
video_df = string_to_dataframe(video)

# Extract probabilities into a DataFrame
probabilities_df = pd.DataFrame(video_df['predicted_probabilities'].tolist())

# Extract predicted labels
predicted_labels = video_df['predicted_label'].tolist()

# Create a figure for plotting
plt.figure(figsize=(10, 6))

# Plot the probabilities for each class
for class_index in probabilities_df.columns:
    class_probabilities = probabilities_df[class_index]
    if class_index == 2:
      plt.plot(class_probabilities, label=f'Background')
    elif class_index == 3:
      plt.plot(class_probabilities, label=f'Grunt')
    elif class_index == 4:
      plt.plot(class_probabilities, label=f'Snaps')
    else:
      plt.plot(class_probabilities, label=f'Class {class_index}')

# Set labels and title
plt.xlabel('Clip Index')
plt.ylabel('Probability')
plt.title('Probabilities for Each Class')
plt.legend()

# Display the plot
plt.show()

print(not_present.iloc[i])

In [ ]:
probabilities_df.columns.to_list()

[0, 1, 2, 3, 4]

In [ ]:
predicted_label = video_df['audio_clip'].iloc[clip_i]


In [ ]:
spreadsheet['Calls'].sum()

39.0

In [ ]:
#@title Label Videos

labels_dir = '/content/drive/My Drive/manacus_v1/labels/'
if not os.path.exists(labels_dir):
  os.mkdir(labels_dir)
labels = [os.path.join(labels_dir, f) for f in os.listdir(labels_dir)]
total = []
for f in labels:
  for i in os.listdir(f):
    total.append('_'.join(i.split('_')[:-1]))
total = set(total)
print(total)
print(len(total), 'videos have been anotated')

nav = BoxNavigator()

for i in range(len(spreadsheet)):
  video_name = spreadsheet.loc[i, 'FileName']
  if "P" in video_name and "L" in video_name and video_name.split('.MP4')[0] not in total:
    nav.download_vid(video_name)
    nav.extract_audio_clips(video_name)
    nav.labelclips(video_name)

{'L6.P2_7.12.22-8.3.22_0223', 'L6.P2_7.12.22-8.3.22_0179', 'L6.P2_10.18.22-11.03.22_0028', 'L6.P2_6.10.22-6.23.22_0014', 'L6.P2_10.18.22-11.03.22_0042', 'L6.P2_7.12.22-8.3.22_0061', 'L6.P2_7.12.22-8.3.22_0171', 'L6.P2_7.12.22-8.3.22_0180', 'L6.P2_10.18.22-11.03.22_0031', 'L6.P2_7.12.22-8.3.22_0059', 'L6.P2_6.10.22-6.23.22_0035', 'L6.P2_7.12.22-8.3.22_0191', 'L6.P2_7.12.22-8.3.22_0192'}
13 videos have been anotated
Mounted at /content/drive/


{'date': 'Tue, 06 Aug 2024 03:55:34 GMT', 'x-envoy-upstream-service-time': '17', 'strict-transport-security': 'max-age=31536000', 'www-authenticate': 'Bearer realm="Service", error="invalid_token", error_description="The access token provided is invalid."', 'box-request-id': '06e39014c19378f879b35fb1187929ced', 'via': '1.1 google', 'Content-Length': '0', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
b''

{'date': 'Tue, 06 Aug 2024 03:55:34 GMT', 'content-type': 'application/json', 'strict-transport-security': 'max-age=31536000', 'set-cookie': 'box_visitor_id=66b19eb696bb51.01966349; expires=Wed, 06-Aug-2025 03:55:34 GMT; Max-Age=31536000; path=/; domain=.box.com; secure; SameSite=None, bv=EUG-7972; expires=Tue, 13-Aug-2024 03:55:34 GMT; Max-Age=604800; path=/; domain=.app.box.com; secure, cn=29; expires=Wed, 06-Aug-2025 03:55:34 GMT; Max-Age=31536000; path=/; domain=.app.box.com; secure, site_preference=desktop; path=/; domain=.box.com; secure', 'cache-control': 'no-stor

Enter your Box Developer Token: L6ElCOJwWQOUHfBX3pQNXlNMvXuaPqtG
    Successfully saved video 'L6.P2_6.10.22-6.23.22_0018.MP4' from Box in Drive.
MoviePy - Writing audio in /content/drive/MyDrive/manacus_v1/Downloaded Videos/L6.P2_6.10.22-6.23.22_0018/L6.P2_6.10.22-6.23.22_0018_0000.wav


MoviePy - Done.
Saved clip: L6.P2_6.10.22-6.23.22_0018_0000.wav
MoviePy - Writing audio in /content/drive/MyDrive/manacus_v1/Downloaded Videos/L6.P2_6.10.22-6.23.22_0018/L6.P2_6.10.22-6.23.22_0018_0001.wav


MoviePy - Done.
Saved clip: L6.P2_6.10.22-6.23.22_0018_0001.wav
MoviePy - Writing audio in /content/drive/MyDrive/manacus_v1/Downloaded Videos/L6.P2_6.10.22-6.23.22_0018/L6.P2_6.10.22-6.23.22_0018_0002.wav


...

MoviePy - Done.
Saved clip: L6.P2_6.10.22-6.23.22_0018_0033.wav
MoviePy - Writing audio in /content/drive/MyDrive/manacus_v1/Downloaded Videos/L6.P2_6.10.22-6.23.22_0018/L6.P2_6.10.22-6.23.22_0018_0034.wav


MoviePy - Done.
Saved clip: L6.P2_6.10.22-6.23.22_0018_0034.wav
MoviePy - Writing audio in /content/drive/MyDrive/manacus_v1/Downloaded Videos/L6.P2_6.10.22-6.23.22_0018/L6.P2_6.10.22-6.23.22_0018_0035.wav


MoviePy - Done.
Saved clip: L6.P2_6.10.22-6.23.22_0018_0035.wav
MoviePy - Writing audio in /content/drive/MyDrive/manacus_v1/Downloaded Videos/L6.P2_6.10.22-6.23.22_0018/L6.P2_6.10.22-6.23.22_0018_0036.wav


chunk:   0%|          | 0/12 [00:00<?, ?it/s, now=None]

KeyboardInterrupt: 